In [0]:
# Define os parâmetros da Job
dbutils.widgets.text('p_path_file_location','')
dbutils.widgets.text('p_layer','')
dbutils.widgets.text('p_file_name_raw','')
dbutils.widgets.text('p_file_type_raw','')
dbutils.widgets.text('p_mode_write_bronze','')

###Premissa: Escolher um conjunto de dados do Kaggle relacionado a vendas. 
Obs.: Certifique-se de que o conjunto de dados inclui informações como datas, produtos, quantidades vendidas, etc.

**Dataset escolhido**: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce/data

In [0]:
# Importando Bibliotecas
from pyspark.sql import DataFrame
from pyspark.sql.functions import current_timestamp

####Funções para leitura, processamento e gravação de dados

In [0]:
def ler_csv(path :str, layer :str, file_name_raw :str, file_type_raw :str) -> DataFrame:
    """
    Lê um arquivo CSV do ADSL Gen2.

    Args:
        layer (str): Camada do arquivo.
        file_name (str): Nome do arquivo.
        file_type (str): Tipo do arquivo.

    Returns:
        DataFrame: O DataFrame resultante da leitura do arquivo CSV.
    """
    try:
        # Opção de leitura do arquivo CSV
        infer_schema = "true"
        first_row_is_header = "true"
        delimiter = ","

        # Lê arquivo csv
        df = spark.read.format(file_type_raw)\
            .option('header',first_row_is_header)\
            .option('inferschema',infer_schema)\
            .load(f'{path}/{layer}/{file_name_raw}.{file_type_raw}')

        return df
    except Exception as e:
        print(f"Erro ao ler o arquivo CSV: {e}")
        return None

In [0]:
def add_timestamp(df:DataFrame) -> DataFrame:
    """
    Adiciona uma coluna com a data e hora atuais a um DataFrame do Spark.

    Args:
        df (DataFrame): DataFrame do Spark.
        nome_coluna (str): Nome da coluna a ser adicionada.

    Returns:
        DataFrame: O DataFrame original com a nova coluna adicionada.
    """
    try:
        # Adiciona uma coluna com a data e hora atuais
        df_timestamp = df.withColumn('row_ingestion_timestamp', current_timestamp())
        
        return df_timestamp
    except Exception as e:
        print(f"Erro ao adicionar timestamp: {e}")
        return None

In [0]:
def gravar_parquet(df :DataFrame, destination_path :str, mode_write_bronze :str) -> None:
    """
    Grava os dados de um DataFrame no formato Parquet.

    Args:
        df (DataFrame): DataFrame do Spark.
        caminho_destino (str): Caminho de destino para salvar o arquivo Parquet.
    """
    # Grava o DataFrame no formato Parquet
    df.write.mode(mode_write_bronze).parquet(destination_path)

    print("Dados salvos com sucesso no formato Parquet.")


In [0]:
def criar_tabela_databricks(file_path_parquet :str, table_name :str) -> None:
    """
    Cria tabela no formato delta no Databricks a partir de um arquivo parquet

    Args:
        file_path_parquet: Caminho da container (pasta) onde se encontra o arquivo parquet
        schema_name: Nome do Schema 
        table_name: Nome da tabela
    """

    # Verifica se a tabela existe
    spark.sql('use catalog poc_datum')
    tabelas_existentes = spark.sql("SHOW TABLES")
    if table_name not in [row.tableName for row in tabelas_existentes.collect()]:
        # Carrega o DataFrame a partir do arquivo parquet
        df = spark.read.parquet(file_path_parquet)

        # Salva o Dataframe como uma tabela no Databricks
        df.write.format('delta').mode("overwrite").saveAsTable(f'{table_name}')

        print(f"A tabela '{table_name}' foi criada com sucesso no Databricks.")
    else:
        print(f"A tabela '{table_name}' já existe no Databricks.")

###Premissa: 
  - Carregue o conjunto de dados no Databricks.
  - Explore o esquema dos dados e faça ajustes conforme necessário.

In [0]:
# Parâmetros de leitura de arquivo
path_file_location = dbutils.widgets.get('p_path_file_location') #"abfss://datum-metastore@proofconceptdatum.dfs.core.windows.net"
layer = dbutils.widgets.get('p_layer') #"raw/olistbr-brazilian-ecommerce"
file_name_raw = dbutils.widgets.get('p_file_name_raw') #"olist_customers_dataset"
file_type_raw = dbutils.widgets.get('p_file_type_raw') #"csv"

# Lê o CSV
df = ler_csv(path=path_file_location, layer=layer, file_name_raw=file_name_raw, file_type_raw=file_type_raw)

# Adiciona coluna row_ingestion_timestamp
df_rits = add_timestamp(df=df)

display(df_rits)

### Premissa: Grave os dados transformados e agregados em um formato Parquet para persistência eficiente.

Obs.: Aqui não estamos considerando opções avançadas de carga de dados, como carga incremental, upsert ou Auto Loader, pois este é um cenário de prova de conceito (POC). Entretanto, é crucial considerar essas estratégias em um ambiente de produção para garantir eficiência e precisão na gestão de dados. Além disso, o particionamento de dados também deve ser cuidadosamente avaliado e implementado para otimizar o desempenho e a escalabilidade do sistema.

In [0]:
# Parâmetros de gravação de arquivo
file_name_bronze = file_name_raw
destination_path = f'{path_file_location}/bronze/olistbr-brazilian-ecommerce/{file_name_bronze}'
mode_write_bronze = dbutils.widgets.get('p_mode_write_bronze') #'overwrite'

# Salva arquivo no ADLS (Azure Data Lake Storage) em formato parquet, na camada bronze
gravar_parquet(df_rits, destination_path=destination_path, mode_write_bronze=mode_write_bronze)